In [133]:
import importlib
import figaro_util
import dataset
import pandas as pd

importlib.reload(figaro_util)
from figaro_util import *   # refreshes all top-level symbols
importlib.reload(dataset)
from dataset import *   # refreshes all top-level symbols
import ast

{TreeNode(value=2)}
{TreeNode(value=3)}
set()
{TreeNode(value=2)}
{TreeNode(value=3), TreeNode(value=4)}
set()
set()
{TreeNode(value=2)}
{TreeNode(value=3), TreeNode(value=4)}
set()
set()
{TreeNode(value=1)}
{TreeNode(value=2)}
{TreeNode(value=3), TreeNode(value=4)}
set()
set()


In [2]:
print(up_result(2, three_node_table_access, three_node_tree, {"X1,2": "b1"}))
print()
print(up_result(2, three_node_table_access, three_node_tree, {"X1,2": "b2"}))
print()
print(up_result(3, three_node_table_access, three_node_tree, {"X2,3": "c1"}))
print()
print(up_result(1, three_node_table_access, three_node_tree, {}))
print()

NameError: name 'three_node_table_access' is not defined

In [136]:
print(down_result(2, three_node_table_access, three_node_tree, {"X2,3": "c1"}))
print()
print(down_result(1, three_node_table_access, three_node_tree, {"X1,2": "b1"}))
print()
print(down_result(1, three_node_table_access, three_node_tree, {"X1,2": "b2"}))
print()

   Y2 X1,2
0  a1   b1
1  a2   b2

  X1,2  Y1 X2,3  Y2  Y3
0   b1  10   c1  a1  d1
1   b1  10   c1  a1  d2

  X1,2  Y1 X2,3  Y2  Y3
0   b2  20   c1  a2  d1
1   b2  20   c1  a2  d2
2   b2  30   c1  a2  d1
3   b2  30   c1  a2  d2



In [137]:
print(else_result(1, three_node_table_access, three_node_tree, {"X1,2": "b1"}))
print()
print(else_result(1, three_node_table_access, three_node_tree, {"X1,2": "b2"}))
print()


print(else_result(2, three_node_table_access, three_node_tree, {"X1,2": "b1", "X2,3": "c1"}))
print()
print(else_result(2, three_node_table_access, three_node_tree, {"X1,2": "b2", "X2,3": "c1"}))
print()

IndexError: list index out of range

# Four node example #

In [4]:
# create root
s1 = TreeNode(1)

s2 = TreeNode(2, parent=s1)

s3 = TreeNode(3, parent=s2)

s4 = TreeNode(4, parent=s2)

# add another child explicitly
print(s1.children)
print(s2.children)
print(s3.children)
print(s4.children)


four_node_tree = {
    1: s1,
    2: s2,
    3: s3,
    4: s4
}

{TreeNode(value=2)}
{TreeNode(value=3), TreeNode(value=4)}
set()
set()


In [5]:
import pandas as pd

four_node_table_access = {
    1: pd.DataFrame({
        "X1,2": ["a1", "a2", "a2", "a3"],
        "Y1":   ["b1", "b2", "b3", "b4"]
    }),
    
    2: pd.DataFrame({
        "X1,2": ["a1", "a1", "a2", "a3"],
        "X2,3": ["c1", "c2", "c2", "c3"],
        "X2,4": ["d1", "d1", "d2", "d3"],
        "Y2":   ["e1", "e2", "e3", "e4"]
    }),
    
    3: pd.DataFrame({
        "X2,3": ["c1", "c1", "c2", "c3"],
        "Y3":   ["f1", "f2", "f3", "f4"]
    }),
    
    4: pd.DataFrame({
        "X2,4": ["d1", "d1", "d2", "d3"],
        "Y4":   ["g1", "g2", "g3", "g4"]
    })
}

In [6]:
print(all_result(1, four_node_table_access, four_node_tree))

  X1,2  Y1 X2,3 X2,4  Y2  Y3  Y4
0   a1  b1   c1   d1  e1  f1  g1
1   a1  b1   c1   d1  e1  f1  g2
2   a1  b1   c1   d1  e1  f2  g1
3   a1  b1   c1   d1  e1  f2  g2
4   a1  b1   c2   d1  e2  f3  g1
5   a1  b1   c2   d1  e2  f3  g2
6   a2  b2   c2   d2  e3  f3  g3
7   a2  b3   c2   d2  e3  f3  g3
8   a3  b4   c3   d3  e4  f4  g4


In [7]:
print(else_result(1, four_node_table_access, four_node_tree, {"X1,2": "a1"}))

  X1,2 X2,3 X2,4  Y2  Y3  Y4
0   a1   c1   d1  e1  f1  g1
1   a1   c1   d1  e1  f1  g2
2   a1   c1   d1  e1  f2  g1
3   a1   c1   d1  e1  f2  g2
4   a1   c2   d1  e2  f3  g1
5   a1   c2   d1  e2  f3  g2


In [8]:
print(else_result(2, four_node_table_access, four_node_tree, {"X1,2": "a1", "X2,3": "c1", "X2,4": "d1"}))

  X1,2  Y1 X2,3 X2,4  Y3  Y4
0   a1  b1   c1   d1  f1  g1
1   a1  b1   c1   d1  f1  g2
2   a1  b1   c1   d1  f2  g1
3   a1  b1   c1   d1  f2  g2


# FIGARO #

In [122]:
def initialise_empty_data(output_column_names):
    Data = pd.DataFrame(columns=output_column_names)
    scales = {}
    return Data, scales
    

In [123]:
def figaro(root_index, index, table_access, tree_access, wrap_singleton=True):
    output_column_names = sorted([f"Y{key}" for key in table_access.keys()])
    Data, scales = initialise_empty_data(output_column_names)
    head_and_tails(Data, scales, index, table_access, tree_access, wrap_singleton)
    current_node = tree_access[index]
    if current_node.children:
        process_and_join_children(Data, scales, root_index, index, table_access, tree_access,  wrap_singleton)
        if index != root_index:
            Data,scales = project_away_join_attributes(Data, scales, index, table_access, tree_access, wrap_singleton)

    print(f"data at index {index}:")
    print(Data)
    return Data, scales
    
    
def head_and_tails(Data, scales, index, table_access, tree_access, wrap_singleton):
    current_table = table_access[index]
    join_key_columns = sorted([column_name for column_name in current_table.columns.tolist() if column_name.startswith("X")])
    output_column = f"Y{index}"

    # Group by join keys to get lists of Y values
    grouped = current_table.groupby(join_key_columns)[output_column].apply(list).reset_index()
    
    # Directly write each row to Data
    for _, row in grouped.iterrows():
        row_index = str(construct_row_index_dict(index, row, join_key_columns))
        head_value = Head(row[output_column], wrap_singleton)
        fill_in_data(Data, row_index, output_column, head_value)
        # print(row[join_key_columns].to_dict())
        # composite_key = f"{', '.join(row[join_key_columns])}"
        if not wrap_singleton and len(row[output_column]) == 1:
            scales[row_index] = "1"
        else:
            scales[row_index] = f"sqrt({len(row[output_column])})"
    
    # return Data, scales

In [124]:
def process_and_join_children(Data, scales, root_index, index, table_access, tree_access, wrap_singleton):
    current_node = tree_access[index]
    current_table = table_access[index]
    join_key_columns = sorted([column_name for column_name in current_table.columns.tolist() if column_name.startswith("X")])

    child_Data_dict = {}
    child_scales_dict = {}
    
    for child in current_node.children:
        child_index = child.value
        child_Data, child_scales = figaro(root_index, child_index, table_access, tree_access, wrap_singleton)
        child_Data_dict[child_index] = child_Data
        child_scales_dict[child_index] = child_scales
        
    # Group by join keys to get lists of Y values
    output_column_i = f"Y{index}"
    grouped = current_table.groupby(join_key_columns)[output_column_i].apply(list).reset_index()
    
    for _, row in grouped.iterrows():
        row_index_dict = construct_row_index_dict(index, row, join_key_columns)
        row_index = str(row_index_dict)
        
        projected_value_i = Data.loc[row_index, output_column_i]
        new_scale_i = scales[row_index]
        
        for child in current_node.children:
            j = child.value
            Data_j = child_Data_dict[j]
            scales_j = child_scales_dict[j]
            
            all_node_indices = get_all_indices_on_subtree(child)
            table_j = table_access[j]
            common_keys_j = find_key(current_table, table_j)
            projected_row_index_dict_j = {key: row_index_dict[key] for key in sorted(common_keys_j) if key in row_index_dict}
            projected_row_index_j = str(projected_row_index_dict_j)
            
            
            for k in all_node_indices:
                output_column_k = f"Y{k}"
                projected_value_k = multiply(scales[row_index], Data_j.loc[projected_row_index_j, output_column_k])
                
                for j_prime in child_scales_dict:
                    if j_prime != j:
                        table_j_prime = table_access[j_prime]
                        scale_j_prime = child_scales_dict[j_prime]
                        common_keys_j_prime = find_key(current_table, table_j_prime)
                        projected_row_index_dict_j_prime = {key: row_index_dict[key] for key in sorted(common_keys_j_prime) if key in row_index_dict}
                        projected_row_index_j_prime = str(projected_row_index_dict_j_prime)
                        projected_value_k = multiply(scale_j_prime[projected_row_index_j_prime], projected_value_k)

                fill_in_data(Data, row_index, output_column_k, projected_value_k)

            
            projected_value_i = multiply(scales_j[projected_row_index_j], projected_value_i)
            new_scale_i = multiply(scales_j[projected_row_index_j], new_scale_i)
        
        fill_in_data(Data, row_index, output_column_i, projected_value_i)
        scales[row_index] = new_scale_i


In [125]:
def project_away_join_attributes(Data, scales, index, table_access, tree_access, wrap_singleton):
    current_node = tree_access[index]
    current_table = table_access[index]

    parent_index = current_node.parent.value
    parent_table = table_access[parent_index]

    common_keys_with_parent = find_key(current_table, parent_table)

    output_column_names = sorted([f"Y{key}" for key in table_access.keys()])
    _, scales_prime = initialise_empty_data(output_column_names)
    
    Data_prime = data_projection(Data, scales, common_keys_with_parent, wrap_singleton)
    
    for key in Data_prime.index:
        key_value_map = ast.literal_eval(key)
        down_result_scale = down_result(index, table_access, tree_access, key_value_map).shape[0]
        if down_result_scale == 1 and not wrap_singleton:
            scales_prime[key] = "1"
        else:
            scales_prime[key] = f"sqrt({down_result_scale})"

    print(f"after projection to {parent_index}")
    print(Data_prime)
    return Data_prime, scales_prime

In [111]:
pd.set_option('display.max_colwidth', None)

In [115]:
Data, scales = figaro(1, 1, three_node_table_access_v1, three_node_tree_v1)
print(Data)

data at index 3:
               Y1 Y2         Y3
{'X2,3': 'c1'}  0  0  H(d1, d2)
after projection to 1
                                                       Y1  \
{'X1,2': 'b1'}  (sqrt(2)*sqrt(1)*0)/Norm(sqrt(2)*sqrt(1))   
{'X1,2': 'b2'}  (sqrt(2)*sqrt(2)*0)/Norm(sqrt(2)*sqrt(2))   

                                                                       Y2  \
{'X1,2': 'b1'}      (sqrt(2)*sqrt(1)*sqrt(2)*H(10))/Norm(sqrt(2)*sqrt(1))   
{'X1,2': 'b2'}  (sqrt(2)*sqrt(2)*sqrt(2)*H(20, 30))/Norm(sqrt(2)*sqrt(2))   

                                                                       Y3  
{'X1,2': 'b1'}          (sqrt(2)*sqrt(1)*H(d1, d2))/Norm(sqrt(2)*sqrt(1))  
{'X1,2': 'b2'}  (sqrt(2)*sqrt(2)*sqrt(2)*H(d1, d2))/Norm(sqrt(2)*sqrt(2))  
data at index 2:
                                                       Y1  \
{'X1,2': 'b1'}  (sqrt(2)*sqrt(1)*0)/Norm(sqrt(2)*sqrt(1))   
{'X1,2': 'b2'}  (sqrt(2)*sqrt(2)*0)/Norm(sqrt(2)*sqrt(2))   

                                                    

In [114]:
Data, scales = figaro(1, 1, three_node_table_access_v2, three_node_tree_v2)
print(Data)

data at index 2:
               Y1     Y2 Y3
{'X1,2': 'b1'}  0  H(a1)  0
{'X1,2': 'b2'}  0  H(a2)  0
data at index 3:
               Y1 Y2         Y3
{'X1,3': 'c1'}  0  0  H(d1, d2)
data at index 1:
                                             Y1                     Y2  \
{'X1,2': 'b1', 'X1,3': 'c1'}      sqrt(2)*H(10)          sqrt(2)*H(a1)   
{'X1,2': 'b2', 'X1,3': 'c1'}  sqrt(2)*H(20, 30)  sqrt(2)*sqrt(2)*H(a2)   

                                             Y3  
{'X1,2': 'b1', 'X1,3': 'c1'}          H(d1, d2)  
{'X1,2': 'b2', 'X1,3': 'c1'}  sqrt(2)*H(d1, d2)  
                                             Y1                     Y2  \
{'X1,2': 'b1', 'X1,3': 'c1'}      sqrt(2)*H(10)          sqrt(2)*H(a1)   
{'X1,2': 'b2', 'X1,3': 'c1'}  sqrt(2)*H(20, 30)  sqrt(2)*sqrt(2)*H(a2)   

                                             Y3  
{'X1,2': 'b1', 'X1,3': 'c1'}          H(d1, d2)  
{'X1,2': 'b2', 'X1,3': 'c1'}  sqrt(2)*H(d1, d2)  


In [95]:
Data, scales = figaro(3, 1, 1, three_node_table_access_v3, three_node_tree_v3)
# print(Data)

data at index 3:
               Y1 Y2     Y3
{'X2,3': 'b1'}  0  0  H(a1)
{'X2,3': 'b2'}  0  0  H(a2)
after projection to 1
                                             Y1  \
{'X1,2': 'c1'}  (1*0+sqrt(2)*0)/Norm(1,sqrt(2))   

                                                         Y2  \
{'X1,2': 'c1'}  (1*H(10)+sqrt(2)*H(20, 30))/Norm(1,sqrt(2))   

                                                             Y3  
{'X1,2': 'c1'}  (1*H(a1)+sqrt(2)*sqrt(2)*H(a2))/Norm(1,sqrt(2))  
data at index 2:
                                             Y1  \
{'X1,2': 'c1'}  (1*0+sqrt(2)*0)/Norm(1,sqrt(2))   

                                                         Y2  \
{'X1,2': 'c1'}  (1*H(10)+sqrt(2)*H(20, 30))/Norm(1,sqrt(2))   

                                                             Y3  
{'X1,2': 'c1'}  (1*H(a1)+sqrt(2)*sqrt(2)*H(a2))/Norm(1,sqrt(2))  
data at index 1:
                               Y1  \
{'X1,2': 'c1'}  sqrt(3)*H(d1, d2)   

                                            

In [132]:
Data, scales = figaro(1, 1, four_node_four_rows_table_access_v1, four_node_four_rows_tree_v1, wrap_singleton=False)
# print(Data)

data at index 3:
               Y1 Y2  Y3 Y4
{'X2,3': 'c1'}  0  0  e1  0
{'X2,3': 'c2'}  0  0  e2  0
data at index 4:
               Y1 Y2 Y3  Y4
{'X2,4': 'd1'}  0  0  0  f1
after projection to 1
                                           Y1  \
{'X1,2': 'b1'}                              0   
{'X1,2': 'b2'}  (sqrt(2)*0+0)/Norm(sqrt(2),1)   

                                                    Y2  \
{'X1,2': 'b1'}                                      10   
{'X1,2': 'b2'}  (sqrt(2)*H(20, 30)+40)/Norm(sqrt(2),1)   

                                                     Y3  \
{'X1,2': 'b1'}                                       e1   
{'X1,2': 'b2'}  (sqrt(2)*sqrt(2)*e1+e2)/Norm(sqrt(2),1)   

                                                     Y4  
{'X1,2': 'b1'}                                       f1  
{'X1,2': 'b2'}  (sqrt(2)*sqrt(2)*f1+f1)/Norm(sqrt(2),1)  
data at index 2:
                                           Y1  \
{'X1,2': 'b1'}                              0   
{'X1,2': 'b2'}

In [134]:
Data, scales = figaro(0, 0, five_node_padded_from_four_table_access_v1, five_node_padded_from_four_tree_v1, wrap_singleton=False)
# print(Data)

data at index 3:
               Y0 Y1 Y2  Y3 Y4
{'X2,3': 'c1'}  0  0  0  e1  0
{'X2,3': 'c2'}  0  0  0  e2  0
data at index 4:
               Y0 Y1 Y2 Y3  Y4
{'X2,4': 'd1'}  0  0  0  0  f1
after projection to 1
                                           Y0                             Y1  \
{'X1,2': 'b1'}                              0                              0   
{'X1,2': 'b2'}  (sqrt(2)*0+0)/Norm(sqrt(2),1)  (sqrt(2)*0+0)/Norm(sqrt(2),1)   

                                                    Y2  \
{'X1,2': 'b1'}                                      10   
{'X1,2': 'b2'}  (sqrt(2)*H(20, 30)+40)/Norm(sqrt(2),1)   

                                                     Y3  \
{'X1,2': 'b1'}                                       e1   
{'X1,2': 'b2'}  (sqrt(2)*sqrt(2)*e1+e2)/Norm(sqrt(2),1)   

                                                     Y4  
{'X1,2': 'b1'}                                       f1  
{'X1,2': 'b2'}  (sqrt(2)*sqrt(2)*f1+f1)/Norm(sqrt(2),1)  
data at index 2:
    